# Bibliotecas

In [1]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import RagTool
import sqlite3
from datetime import datetime

# Credenciais

In [2]:
os.environ['OPENAI_API_KEY'] = os.getenv('api_key')
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"
model = os.environ["OPENAI_MODEL_NAME"]

# SQLite

In [ ]:
DB_FILE = "../db/historico.db"

def setup_database():
    """
    Cria o banco de dados e a tabela.
    """
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS interacoes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT NOT NULL, 
            prompt_usuario TEXT NOT NULL,
            resposta_agente TEXT NOT NULL
        )
    """)
    conn.commit()
    conn.close()

def log_interaction(prompt, response):
    """
    Salva a interação no banco de dados.
    """
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    timestamp_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    cursor.execute("""
        INSERT INTO interacoes (timestamp, prompt_usuario, resposta_agente)
        VALUES (?, ?, ?)
    """, (timestamp_str, prompt, response))
    
    conn.commit()
    conn.close()
    print("\n[LOG] Interação registrada com sucesso no banco de dados.")

# RAG

In [4]:
# RagTool
knowledge_base = RagTool(
    rag_name="Busca em assuntos de SAC",
    docs=['../data/dados-sac.md'],
    description="Uma ferramenta para buscar informações específicas dentro do documento de SAC da C&A para tratar de assuntos de políticas de troca, devolução etc.",
    config={
    "vectordb": {
        "config": {
            "dir": "../db",  # Specify custom database directory
        }
    }
})

# Agente SAC
agente_sac_cea = Agent(
  role='Especialista de Atendimento ao Cliente C&A',
  goal='Fornecer informações claras, precisas e amigáveis sobre as políticas de troca, devolução e reembolso da C&A, com base estritamente no documento de políticas oficial.',
  backstory=(
      "Você é o assistente virtual da C&A, treinado para ajudar os clientes a navegar por nossas políticas de pós-venda. "
      "Sua prioridade é a satisfação do cliente, garantindo que cada resposta seja útil, empática e 100% alinhada "
      "com as regras da companhia. Você é paciente e sabe como encontrar a informação certa para cada caso."
  ),
  tools=[knowledge_base], # Tool para RAG
  llm=model,
  allow_delegation=False
)

# Task agente SAC
tarefa_sac_rag = Task(
  description=(
      "Um cliente tem uma dúvida sobre: '{topic}'. "
      "Use a ferramenta de consulta de políticas para encontrar a informação exata e formule uma resposta clara e amigável. "
      "Seja prestativo e guie o cliente sobre os próximos passos, se houver."
  ),
  expected_output='Uma resposta clara e completa para a pergunta original ({topic}), baseada estritamente no conteúdo do documento.',
  agent=agente_sac_cea
)

# Arquitetura Crew
rag_sac = Crew(
  agents=[agente_sac_cea],
  tasks=[tarefa_sac_rag],
  process=Process.sequential
)

# Garante integridade do banco de dados
setup_database()

# Prompt do usuário
question = "Tive um problema com o meu pedido, como faço a devolução"
result = rag_sac.kickoff(inputs={'topic': question})

log_interaction(prompt=question, response=str(result))

print("\n\n########################")
print("## Resultado:")
print("########################\n")
print(result)

c:\Users\guilh\GitHub\llm-generative-ai-applications\cea-rag-and-catalog-search\venv-cea-llm-case\lib\site-packages\alembic\config.py:577: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(
c:\Users\guilh\GitHub\llm-generative-ai-applications\cea-rag-and-catalog-search\venv-cea-llm-case\lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
c:\Users\guilh\GitHub\llm-generative-ai-applications\cea-rag-and-catalog-search\venv-cea-llm-case\lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Ins


[LOG] Interação registrada com sucesso no banco de dados.


########################
## Resultado:
########################

Para realizar a devolução de um produto, você deve seguir os seguintes passos: 

1. **Prazo para Devolução**: Você pode solicitar a devolução em até 30 dias corridos após a data de recebimento do produto.

2. **Condições para Devolução**: O produto deve estar em sua embalagem original, sem indícios de uso, com todas as etiquetas e acessórios que o acompanham.

3. **Como Proceder**:
   - Acesse o site da C&A e entre na sua conta.
   - Vá até a opção de "Meus Pedidos".
   - Selecione o pedido que deseja devolver e clique na opção de devolução.
   - Siga as instruções para imprimir a etiqueta de devolução.

4. **Envio do Produto**: Coloque o produto na embalagem original, feche-a adequadamente e coloque a etiqueta de devolução na caixa. Você pode enviar o produto pelo Correios ou, se disponível, optar pelo serviço de coleta.

5. **Reembolso**: Após recebermos o pro

# Consulta SQLite

In [ ]:
conn = sqlite3.connect('../db/historico.db')
cursor = conn.cursor()

print("--- Histórico de Interações ---")
for row in cursor.execute("SELECT id, timestamp, prompt_usuario, resposta_agente FROM interacoes ORDER BY timestamp DESC"):
    print(f"\nID: {row[0]}")
    print(f"Data/Hora: {row[1]}")
    print(f"Usuário Perguntou: {row[2]}")
    print(f"Agente Respondeu: {row[3]}")
    print("-" * 20)

conn.close()

--- Histórico de Interações ---

ID: 9
Data/Hora: 2025-06-25 20:08:26
Usuário Perguntou: Tive um problema com o meu pedido, como faço a devolução
Agente Respondeu: Para realizar a devolução de um produto, você deve seguir os seguintes passos: 

1. **Prazo para Devolução**: Você pode solicitar a devolução em até 30 dias corridos após a data de recebimento do produto.

2. **Condições para Devolução**: O produto deve estar em sua embalagem original, sem indícios de uso, com todas as etiquetas e acessórios que o acompanham.

3. **Como Proceder**:
   - Acesse o site da C&A e entre na sua conta.
   - Vá até a opção de "Meus Pedidos".
   - Selecione o pedido que deseja devolver e clique na opção de devolução.
   - Siga as instruções para imprimir a etiqueta de devolução.

4. **Envio do Produto**: Coloque o produto na embalagem original, feche-a adequadamente e coloque a etiqueta de devolução na caixa. Você pode enviar o produto pelo Correios ou, se disponível, optar pelo serviço de coleta.

5